In [ ]:
#import neccesary libraries

In [51]:
from fastapi import FastAPI, HTTPException, Query, Path, Depends
from pydantic import BaseModel
from typing import List, Optional

In [52]:
app = FastAPI()

In [53]:
fake_db = []

In [54]:
# Book Model

In [55]:
class Book(BaseModel):
    id: int
    title: str
    author: str
    description: Optional[str] = None
    rating: Optional[float] = None

In [56]:
# Dependency: Simulating a database check

In [57]:
def get_db():
    if not fake_db:
        raise HTTPException(status_code=404, detail="No books found")
    return fake_db

In [58]:
# Route to list all books

In [59]:
@app.get("/books/", response_model=List[Book])
def list_books(db=Depends(get_db)):
    return db

In [60]:
# Route to get a specific book by ID

In [61]:
@app.get("/books/{book_id}", response_model=Book)
def get_book(book_id: int = Path(..., description="ID of the book to retrieve")):
    for book in fake_db:
        if book["id"] == book_id:
            return book
    raise HTTPException(status_code=404, detail="Book not found")

In [62]:
# Route to search books by title or author

In [63]:
@app.get("/books/search/", response_model=List[Book])
def search_books(
    title: Optional[str] = Query(None, description="Search by book title"),
    author: Optional[str] = Query(None, description="Search by author name"),
    db=Depends(get_db),
):
    results = [book for book in db if (title and title.lower() in book["title"].lower()) or
               (author and author.lower() in book["author"].lower())]
    if not results:
        raise HTTPException(status_code=404, detail="No matching books found")
    return results

In [64]:
# Route to add a new book

In [65]:
@app.post("/books/", response_model=Book, status_code=201)
def add_book(book: Book):
    # Check if book ID already exists
    if any(existing_book["id"] == book.id for existing_book in fake_db):
        raise HTTPException(status_code=400, detail="Book with this ID already exists")
    fake_db.append(book.dict())
    return book

In [66]:
# Route to update a book

In [67]:
@app.put("/books/{book_id}", response_model=Book)
def update_book(book_id: int, updated_book: Book):
    for idx, book in enumerate(fake_db):
        if book["id"] == book_id:
            fake_db[idx] = updated_book.dict()
            return updated_book
    raise HTTPException(status_code=404, detail="Book not found")

In [68]:
# Route to delete a book

In [69]:
@app.delete("/books/{book_id}", status_code=204)
def delete_book(book_id: int):
    for book in fake_db:
        if book["id"] == book_id:
            fake_db.remove(book)
            return
    raise HTTPException(status_code=404, detail="Book not found")

In [70]:
# Sample Data Loader Route

In [71]:
@app.post("/books/load_sample/")
def load_sample_books():
    sample_books = [
        {"id": 1, "title": "1984", "author": "George Orwell", "description": "Dystopian novel", "rating": 4.8},
        {"id": 2, "title": "To Kill a Mockingbird", "author": "Harper Lee", "description": "Classic novel", "rating": 4.9},
        {"id": 3, "title": "The Great Gatsby", "author": "F. Scott Fitzgerald", "description": "1920s novel", "rating": 4.7}
    ]
    fake_db.extend(sample_books)
    return {"message": "Sample books loaded successfully!"}